In [ ]:
import os
import numpy as np
import tensorflow as tf

# training
from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder, TimingCallback

# testing

print("All packages imported!")

## Training

Example of training with motion classifier.

### Config

In [ ]:
config_fp = '/home/jonathan/git/pdiot-ml/config/train/motion.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/motion/'   
print(cfg)

### Data

In [ ]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                            cfg.TRAIN.DATA.overlap_size, 
                                            cfg.DATASET.num_classes)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                      cfg.TRAIN.DATA.overlap_size,
                                      cfg.DATASET.num_classes)

### Optimization

In [ ]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', cfg.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg.TRAIN.LEN.early_stop)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(cfg.TRAIN.path, 'weights.hdf5'),
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

timing_callback = TimingCallback()

history_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(cfg.TRAIN.path, 'history.csv'), 
    separator=',', 
    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg.TRAIN.LEN.num_epoch, 
          batch_size=cfg.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback,
                     checkpoint_callback,
                     timing_callback,
                     history_callback])

## Testing

Example of LOO testing for task 1 with student 29. 

### Config

### Data

### Optimization

### Evaluation